In [186]:
import pandas as pd
train = pd.read_csv('/kaggle/input/titanic2/train.csv')
test = pd.read_csv('/kaggle/input/titanic2/test.csv')
# len(train)
# train.info()
train.describe()
# train.isnull().mean()*100 
# test.isnull().sum()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [187]:

def handleMissingData(data):
    data['Age'] = pd.to_numeric(data['Age'], errors = 'coerce')
    data['Age'] = data['Age'].fillna(data['Age'].median())

    data['Fare'] = data['Fare'].fillna(data['Fare'].median())
    data['Embarked'].mode()
    # print(type(data['Fare'][0]))

    data.drop(columns = 'Cabin', inplace = True, errors = 'ignore')
    return data
train = handleMissingData(train)
test = handleMissingData(test)
# test.isnull().sum()
train.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [188]:
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

train["Sex"] = train["Sex"].map({"male": 0, "female": 1})

train["Fare"] = np.log1p(train["Fare"])

features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

X = train[feature]
y = train['Survived']
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaler,y, train_size = 0.8, random_state = 0, stratify = y)


In [189]:
# import seaborn as sns 
# import matplotlib.pyplot as plt

# for column in train:
#     sns.boxplot(x = train[column])
#     plt.show()

In [190]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
model = KNeighborsClassifier(
    n_neighbors = 3,
    weights = 'distance',
    metric = 'euclidean'
)
param_grid = {
    'n_neighbors': range(2,12),
    'weights': ['distance', 'uniform'],
    "metric": ["euclidean" ]
}
grid_search = GridSearchCV(model, param_grid, cv = 5 )
grid_search.fit(X_train, y_train)



GridSearchCV(cv=5,
             estimator=KNeighborsClassifier(metric='euclidean', n_neighbors=3,
                                            weights='distance'),
             param_grid={'metric': ['euclidean'], 'n_neighbors': range(2, 12),
                         'weights': ['distance', 'uniform']})

In [191]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = grid_search.predict(X_test)

print(grid_search.best_score_)
print(grid_search.best_params_)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}"  )
print(classification_report(y_test, y_pred))

0.7106470993794938
{'metric': 'euclidean', 'n_neighbors': 10, 'weights': 'uniform'}
Accuracy: 0.72
              precision    recall  f1-score   support

           0       0.72      0.87      0.79       110
           1       0.70      0.46      0.56        69

    accuracy                           0.72       179
   macro avg       0.71      0.67      0.67       179
weighted avg       0.71      0.72      0.70       179

